# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [75]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [76]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [77]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [78]:
data['year1ChangePercent']

0.3525881436563923

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [79]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [80]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
        [
            symbol,
            data[symbol]['price'],
            data[symbol]['stats']['year1ChangePercent'],
            'N/A'
        ],index = my_columns),
            ignore_index = True
        )
final_dataframe


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,160.910,0.391384,N/A
1,AAL,18.614,0.122463,N/A
2,AAP,250.610,0.551846,N/A
3,AAPL,182.430,0.351671,N/A
4,ABBV,136.590,0.353721,N/A
...,...,...,...,...
500,YUM,139.920,0.296107,N/A
501,ZBH,127.468,-0.160845,N/A
502,ZBRA,602.010,0.570281,N/A
503,ZION,64.100,0.516882,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [81]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,83.420,2.342981,N/A
1,148,DVN,45.990,1.893833,N/A
2,299,MCHP,87.240,1.641531,N/A
3,317,MRO,16.440,1.436506,N/A
4,195,FTNT,373.100,1.433005,N/A
5,174,F,20.850,1.351457,N/A
6,175,FANG,112.950,1.261231,N/A
7,345,NVDA,305.060,1.259775,N/A
8,344,NUE,114.590,1.194033,N/A
9,251,IT,347.670,1.138575,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [82]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Please enter size of your portfolio')
    try:
        float(portfolio_size)
    except:
        print('that is not a number. \n please try again')
        portfolio_size = input('Please enter size of your portfolio')
portfolio_input()

Please enter size of your portfolio10000000


In [ ]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(len(final_dataframe.index)):
    final_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i,'Price'])
final_dataframe

## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [162]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'hqm_score'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
        [
            symbol,
            data[symbol]['price'],
            'N/A',
            data[symbol]['stats']['year1ChangePercent'],
            'N/A',
            data[symbol]['stats']['month6ChangePercent'],
            'N/A',
            data[symbol]['stats']['month3ChangePercent'],
            'N/A',
            data[symbol]['stats']['month1ChangePercent'],
            'N/A',
            'N/A'
        ],index = hqm_columns),
            ignore_index = True
        )
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,hqm_score
0,A,165.05,N/A,0.367217,N/A,0.083014,N/A,0.014914,N/A,0.060315,N/A,N/A
1,AAL,18.46,N/A,0.145589,N/A,-0.160769,N/A,-0.129595,N/A,0.015322,N/A,N/A
2,AAP,246.65,N/A,0.547257,N/A,0.184883,N/A,0.156234,N/A,0.091584,N/A,N/A
3,AAPL,181.89,N/A,0.354836,N/A,0.304087,N/A,0.259579,N/A,0.074444,N/A,N/A
4,ABBV,136.90,N/A,0.339503,N/A,0.232798,N/A,0.272855,N/A,0.176499,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,144.98,N/A,0.309373,N/A,0.216987,N/A,0.142080,N/A,0.133240,N/A,N/A
501,ZBH,130.93,N/A,-0.175609,N/A,-0.217474,N/A,-0.136224,N/A,0.064400,N/A,N/A
502,ZBRA,601.00,N/A,0.563058,N/A,0.124630,N/A,0.157038,N/A,0.011106,N/A,N/A
503,ZION,64.65,N/A,0.510603,N/A,0.215067,N/A,0.027073,N/A,0.001278,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [163]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        percentile_column = f'{time_period} Return Percentile'
        change_column = f'{time_period} Price Return'
        hqm_dataframe[change_column] = hqm_dataframe[change_column].astype(float)
        hqm_dataframe.loc[row, percentile_column] = score(hqm_dataframe[change_column], hqm_dataframe.loc[row,change_column])/100
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,hqm_score
0,A,165.05,N/A,0.367217,0.623762,0.083014,0.483168,0.014914,0.261386,0.060315,0.429703,N/A
1,AAL,18.46,N/A,0.145589,0.314851,-0.160769,0.071287,-0.129595,0.041584,0.015322,0.152475,N/A
2,AAP,246.65,N/A,0.547257,0.823762,0.184883,0.714851,0.156234,0.691089,0.091584,0.667327,N/A
3,AAPL,181.89,N/A,0.354836,0.603960,0.304087,0.902970,0.259579,0.900990,0.074444,0.538614,N/A
4,ABBV,136.90,N/A,0.339503,0.596040,0.232798,0.821782,0.272855,0.920792,0.176499,0.968317,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,144.98,N/A,0.309373,0.564356,0.216987,0.784158,0.142080,0.647525,0.133240,0.865347,N/A
501,ZBH,130.93,N/A,-0.175609,0.037624,-0.217474,0.035644,-0.136224,0.035644,0.064400,0.461386,N/A
502,ZBRA,601.00,N/A,0.563058,0.833663,0.124630,0.588119,0.157038,0.693069,0.011106,0.142574,N/A
503,ZION,64.65,N/A,0.510603,0.794059,0.215067,0.780198,0.027073,0.304950,0.001278,0.104950,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [164]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row,'hqm_score'] = mean(momentum_percentiles)
    
hqm_dataframe
    

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,hqm_score
0,A,165.05,N/A,0.367217,0.623762,0.083014,0.483168,0.014914,0.261386,0.060315,0.429703,0.449505
1,AAL,18.46,N/A,0.145589,0.314851,-0.160769,0.071287,-0.129595,0.041584,0.015322,0.152475,0.145050
2,AAP,246.65,N/A,0.547257,0.823762,0.184883,0.714851,0.156234,0.691089,0.091584,0.667327,0.724257
3,AAPL,181.89,N/A,0.354836,0.603960,0.304087,0.902970,0.259579,0.900990,0.074444,0.538614,0.736634
4,ABBV,136.90,N/A,0.339503,0.596040,0.232798,0.821782,0.272855,0.920792,0.176499,0.968317,0.826733
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,144.98,N/A,0.309373,0.564356,0.216987,0.784158,0.142080,0.647525,0.133240,0.865347,0.715347
501,ZBH,130.93,N/A,-0.175609,0.037624,-0.217474,0.035644,-0.136224,0.035644,0.064400,0.461386,0.142574
502,ZBRA,601.00,N/A,0.563058,0.833663,0.124630,0.588119,0.157038,0.693069,0.011106,0.142574,0.564356
503,ZION,64.65,N/A,0.510603,0.794059,0.215067,0.780198,0.027073,0.304950,0.001278,0.104950,0.496040


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [165]:
hqm_dataframe.sort_values('hqm_score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop = True,inplace = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,hqm_score
0,ANET,144.46,N/A,1.014831,0.970297,0.615259,0.988119,0.683879,0.992079,0.164375,0.950495,0.975248
1,CF,71.96,N/A,0.902081,0.966337,0.405412,0.954455,0.287097,0.932673,0.172360,0.962376,0.953960
2,EXR,231.41,N/A,1.032453,0.972277,0.411743,0.962376,0.365914,0.964356,0.146349,0.910891,0.952475
3,AVGO,666.83,N/A,0.582327,0.847525,0.426624,0.970297,0.391497,0.970297,0.216129,0.984158,0.943069
4,ACN,434.16,N/A,0.615273,0.881188,0.430224,0.972277,0.307572,0.946535,0.166167,0.954455,0.938614
5,AZO,2150.65,N/A,0.782632,0.950495,0.407247,0.956436,0.242702,0.875248,0.157142,0.934653,0.929208
6,PLD,174.40,N/A,0.738857,0.942574,0.425248,0.966337,0.364508,0.962376,0.126117,0.829703,0.925248
7,JNPR,36.64,N/A,0.642390,0.895050,0.338594,0.932673,0.316935,0.950495,0.147940,0.916832,0.923762
8,DRE,67.08,N/A,0.685414,0.918812,0.404285,0.952475,0.393975,0.972277,0.127743,0.839604,0.920792
9,PSA,378.80,N/A,0.690611,0.922772,0.268749,0.875248,0.276797,0.928713,0.154000,0.930693,0.914356


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [166]:
portfolio_input()

Please enter size of your portfolio1000000


In [167]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in range(len(hqm_dataframe.index)):
    hqm_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i,'Price'])
hqm_dataframe

/Users/jacobhatef/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1719: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,hqm_score
0,ANET,144.46,138,1.014831,0.970297,0.615259,0.988119,0.683879,0.992079,0.164375,0.950495,0.975248
1,CF,71.96,277,0.902081,0.966337,0.405412,0.954455,0.287097,0.932673,0.172360,0.962376,0.953960
2,EXR,231.41,86,1.032453,0.972277,0.411743,0.962376,0.365914,0.964356,0.146349,0.910891,0.952475
3,AVGO,666.83,29,0.582327,0.847525,0.426624,0.970297,0.391497,0.970297,0.216129,0.984158,0.943069
4,ACN,434.16,46,0.615273,0.881188,0.430224,0.972277,0.307572,0.946535,0.166167,0.954455,0.938614
5,AZO,2150.65,9,0.782632,0.950495,0.407247,0.956436,0.242702,0.875248,0.157142,0.934653,0.929208
6,PLD,174.40,114,0.738857,0.942574,0.425248,0.966337,0.364508,0.962376,0.126117,0.829703,0.925248
7,JNPR,36.64,545,0.642390,0.895050,0.338594,0.932673,0.316935,0.950495,0.147940,0.916832,0.923762
8,DRE,67.08,298,0.685414,0.918812,0.404285,0.952475,0.393975,0.972277,0.127743,0.839604,0.920792
9,PSA,378.80,52,0.690611,0.922772,0.268749,0.875248,0.276797,0.928713,0.154000,0.930693,0.914356


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [168]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine = 'xlsxwriter')
hqm_dataframe.to_excel(writer,sheet_name = 'Momentum Strategy',index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [169]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [170]:
column_formats = {
    'A':['Ticker',string_template],
    'B':['Price',dollar_template],
    'C':['Number of Shares to Buy',integer_template],
    'D':['One-Year Price Return',dollar_template],
    'E':['One-Year Return Percentile',percent_template],
    'F':['Six-Month Price Return',dollar_template],
    'G':['Six-Month Return Percentile',percent_template],
    'H':['Three-Month Price Return',dollar_template],
    'I':['Three-Month Return Percentile',percent_template],
    'J':['One-Month Price Return',dollar_template],
    'K':['One-Month Return Percentile',percent_template],
    'L':['hqm_score',percent_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column( f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
    

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [171]:
writer.save()